In [1]:
# cd ..

/root/anindya/Submission/text2sql/text2sql


In [2]:
from premsql.datasets import (
    BirdDataset,
    SpiderUnifiedDataset,
    DomainsDataset,
    GretelAIDataset
)

from premsql.evaluator.from_sqlite import SQLiteExecutor
from premsql.datasets import Text2SQLDataset
from premsql.tuner.peft import Text2SQLPeftTuner
from premsql.datasets.error_dataset import ErrorDatasetGenerator

/root/miniconda3/envs/deep/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-09-07 10:41:00,820] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/root/miniconda3/envs/deep/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/root/miniconda3/envs/deep/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/root/miniconda3/envs/deep/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


In [3]:
path = "/root/anindya/text2sql/data"
model_name_or_path = "premai-io/prem-1B-SQL"

In [4]:
bird_train = BirdDataset(split="train", dataset_folder=path).setup_dataset(
    num_rows=100,
    model_name_or_path=model_name_or_path
)

2024-09-07 10:41:16,255 - [BIRD-DATASET] - INFO - Loaded Bird Dataset
2024-09-07 10:41:16,257 - [BIRD-DATASET] - INFO - Setting up Bird Dataset
Applying prompt: 100%|██████████| 100/100 [00:00<00:00, 3519.80it/s]
2024-09-07 10:41:16,891 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-07 10:41:16,892 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 100/100 [00:00<00:00, 199.78it/s]


In [5]:
spider_train = SpiderUnifiedDataset(split="train", dataset_folder="./data").setup_dataset(
    num_rows=100,
    model_name_or_path=model_name_or_path
)

2024-09-07 10:41:44,000 - [SPIDER-DATASET] - INFO - Loaded Spider Dataset
2024-09-07 10:41:44,005 - [SPIDER-DATASET] - INFO - Setting up Spider Dataset
Applying prompt: 100%|██████████| 100/100 [00:00<00:00, 4144.69it/s]
2024-09-07 10:41:44,636 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-07 10:41:44,637 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 100/100 [00:00<00:00, 436.83it/s]


In [6]:
domains_dataset = DomainsDataset(split="train", dataset_folder="./data").setup_dataset(
    num_rows=100,
    model_name_or_path=model_name_or_path
)

2024-09-07 10:42:00,249 - [DOMAINS-DATASET] - INFO - Loaded Domains Dataset
2024-09-07 10:42:00,252 - [DOMAINS-DATASET] - INFO - Setting up Domains Dataset
Applying prompt: 100%|██████████| 100/100 [00:00<00:00, 2671.91it/s]
2024-09-07 10:42:00,681 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-07 10:42:00,682 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 100/100 [00:00<00:00, 241.73it/s]


In [7]:
gertelai_dataset = GretelAIDataset(split="train", dataset_folder="./data",).setup_dataset(
    num_rows=100,
    model_name_or_path=model_name_or_path
)

Applying prompt: 100%|██████████| 100/100 [00:00<00:00, 162130.03it/s]
2024-09-07 10:42:14,958 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-07 10:42:14,958 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 100/100 [00:00<00:00, 579.19it/s]


In [8]:
existing_error_dataset = ErrorDatasetGenerator.from_existing(
    experiment_name="testing_error_gen",
    tokenize_model_name_or_path=model_name_or_path
)

2024-09-07 10:42:28,011 - [DATASET] - INFO - Casted dataset with model chat template
2024-09-07 10:42:28,012 - [DATASET] - INFO - Starting Tokenization ...
Tokenizing: 100%|██████████| 10/10 [00:00<00:00, 180.55it/s]


In [9]:
merged_dataset = [
    *spider_train,
    *bird_train,
    *domains_dataset,
    *gertelai_dataset,
    *existing_error_dataset
    
]

In [10]:
bird_dev = Text2SQLDataset(dataset_name="bird", split="validation", dataset_folder=path).setup_dataset(
    num_rows=10,
    filter_by=("difficulty", "challenging")
)

2024-09-07 10:43:00,302 - [BIRD-DATASET] - INFO - Loaded Bird Dataset
2024-09-07 10:43:00,303 - [BIRD-DATASET] - INFO - Setting up Bird Dataset
Applying prompt: 100%|██████████| 10/10 [00:00<00:00, 1762.53it/s]


In [11]:
tuner = Text2SQLPeftTuner(
    model_name_or_path=model_name_or_path,
    experiment_name="lora_tuning"
)

2024-09-07 10:43:05,347 - [LORA-FINETUNE] - WARNING - Setting up Pretrained-Model: premai-io/prem-1B-SQL
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.82s/it]


In [ ]:
tuner.train(
    train_datasets=merged_dataset,
    output_dir="./output",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    evaluation_dataset=bird_dev,
    eval_steps=100,
    max_seq_length=1024,
    executor=SQLiteExecutor(),
    filter_eval_results_by=("difficulty", "challenging")
)